In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import local.lib.mlutils
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

/content/local/lib/mlutils.py:445: SyntaxWarning: invalid escape sequence '\m'
  time_str = "fit time  (TRAIN)    %.2f $\mu$secs"%(fitting_time*1000)


In [ ]:
!pip install opendatasets
import opendatasets as od

In [ ]:
dataset_link="https://www.kaggle.com/competitions/udea-ai-4-eng-20252-pruebas-saber-pro-colombia/overview"
od.download(dataset_link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: anamariagranadarodas
Your Kaggle Key: ··········


100%|██████████| 29.9M/29.9M [00:00<00:00, 1.22GB/s]


Extracting archive ./udea-ai-4-eng-20252-pruebas-saber-pro-colombia/udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip to ./udea-ai-4-eng-20252-pruebas-saber-pro-colombia


In [ ]:
train = pd.read_csv("udea-ai-4-eng-20252-pruebas-saber-pro-colombia/train.csv")
print (train.shape)
train

(692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto,0.237,0.271,0.271,0.311
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,...,N,No,Si,Si,Secundaria (Bachillerato) incompleta,bajo,0.314,0.240,0.278,0.260
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.286,0.240,0.314,0.287
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,...,N,Si,Si,No,Primaria completa,bajo,0.132,0.426,0.261,0.328




---



In [ ]:
# .info() es vital para ver tipos de datos (dtypes) y valores nulos
print("--- Información de 'train' ---")
train.info()

print("\n--- Valores Nulos en 'train' ---")
# Ver un conteo de cuántos nulos hay por columna
print(train.isnull().sum())

--- Información de 'train' ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO      

In [ ]:
print("Separando X (features) y y (target)...")

# 1. Definimos nuestra columna objetivo
target_col = 'RENDIMIENTO_GLOBAL'

# 2. Creamos 'y' (nuestro target)
y = train[target_col]

# 3. Creamos 'X' (nuestras features)
# Quitamos la columna objetivo y el ID, que no es un feature útil
X = train.drop([target_col, 'ID'], axis=1)

print(f"Dimensiones de X (features): {X.shape}")
print(f"Dimensiones de y (target): {y.shape}")

Separando X (features) y y (target)...
Dimensiones de X (features): (692500, 19)
Dimensiones de y (target): (692500,)


In [ ]:
# El target 'RENDIMIENTO_GLOBAL' es categórico y ORDINAL
# (tiene un orden: bajo < medio-bajo < medio-alto < alto)
# Usamos OrdinalEncoder para convertirlo a números (0, 1, 2, 3)

# 1. Definimos el orden correcto de las categorías
categories_order = [['bajo', 'medio-bajo', 'medio-alto', 'alto']]

# 2. Creamos el codificador
target_encoder = OrdinalEncoder(categories=categories_order)

# 3. Ajustamos y transformamos 'y'
# .fit_transform espera un DataFrame (2D), por eso usamos y.to_frame()
y_encoded = target_encoder.fit_transform(y.to_frame())

print(f"Categorías del target: {target_encoder.categories_}")
print(f"Forma de y_encoded: {y_encoded.shape}")
print(f"Ejemplo de 'y' original:\n{y.head(3)}")
print(f"Ejemplo de 'y' codificado:\n{y_encoded[:3]}")

Categorías del target: [array(['bajo', 'medio-bajo', 'medio-alto', 'alto'], dtype=object)]
Forma de y_encoded: (692500, 1)
Ejemplo de 'y' original:
0    medio-alto
1          bajo
2          bajo
Name: RENDIMIENTO_GLOBAL, dtype: object
Ejemplo de 'y' codificado:
[[2.]
 [0.]
 [0.]]


In [ ]:
# Esta es la parte más importante.
# Separamos las columnas por tipo para aplicarles distintas transformaciones.

# 1. Identificar columnas numéricas
# Usamos select_dtypes para tomar todas las columnas que sean números
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
print(f"Columnas numéricas ({len(numeric_features)}): {numeric_features}")

# 2. Identificar columnas categóricas
# Tomamos todas las que sean 'object' (texto)
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# PERIODO_ACADEMICO es numérico (20212) pero debe tratarse como categoría
# Lo movemos de la lista numérica a la categórica
if 'PERIODO_ACADEMICO' in numeric_features:
    numeric_features.remove('PERIODO_ACADEMICO')
    categorical_features.append('PERIODO_ACADEMICO')
    # Lo convertimos a string para que OneHotEncoder lo tome bien
    X['PERIODO_ACADEMICO'] = X['PERIODO_ACADEMICO'].astype(str)

print(f"Columnas categóricas ({len(categorical_features)}): {categorical_features}")

# 3. Crear el pipeline para features NUMÉRICAS
# Un "pipeline" es una secuencia de pasos
numeric_transformer = Pipeline(steps=[
    # Paso 1: Imputar (rellenar) valores faltantes (NaN) con la mediana
    ('imputer', SimpleImputer(strategy='median')),
    # Paso 2: Escalar los datos (importante para redes neuronales)
    ('scaler', StandardScaler())
])

# 4. Crear el pipeline para features CATEGÓRICAS
categorical_transformer = Pipeline(steps=[
    # Paso 1: Imputar valores faltantes con el valor más frecuente
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # Paso 2: Aplicar One-Hot Encoding (convertir "Bogotá", "Antioquia" en columnas 0/1)
    # handle_unknown='ignore' es clave para que no falle si en 'test' aparece una categoría nueva
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Columnas numéricas (5): ['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
Columnas categóricas (15): ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'PERIODO_ACADEMICO']


In [ ]:
# ColumnTransformer aplica los pipelines anteriores a las columnas correctas

# 1. Creamos el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Dejar pasar cualquier columna que no listamos
)

# 2. Aplicamos el preprocesador
# Con .fit_transform() APRENDE de 'X' (train) y lo TRANSFORMA
print("Ajustando y transformando X (train)...")
X_processed = preprocessor.fit_transform(X)

# La salida será una matriz dispersa (sparse matrix) para ahorrar memoria
# Si quieres verla como array de numpy (¡¡CUIDADO, PUEDE USAR MUCHA RAM!!),
# usa: X_processed.toarray()

print("\n¡Preprocesado completado!")
print(f"Dimensiones de X (train) procesado: {X_processed.shape}")

Ajustando y transformando X (train)...

¡Preprocesado completado!
Dimensiones de X (train) procesado: (692500, 1050)


In [ ]:
print("Creando DataFrame con datos procesados para visualización...")

# 1. Obtener los nombres de todas las columnas generadas por el preprocesador
# (Esto incluye las numéricas escaladas y las categóricas post-OneHot)
try:
    # get_feature_names_out() es la forma moderna y recomendada
    # Añadirá prefijos como 'num__' y 'cat__'
    feature_names = preprocessor.get_feature_names_out()

except AttributeError:
    # Plan B por si la versión de sklearn es más antigua (menos probable en Colab)
    print("Usando método alternativo para obtener nombres de columnas...")

    # Nombres de features numéricas (se mantienen igual)
    numeric_cols = numeric_features

    # Nombres de features categóricas (generadas por OneHot)
    # Accedemos al paso 'onehot' dentro del pipeline 'cat'
    onehot_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
    categorical_cols = onehot_encoder.get_feature_names_out(categorical_features)

    # Unir todas las listas de nombres en el orden correcto
    feature_names = numeric_cols + list(categorical_cols)

# 2. Convertir la matriz (posiblemente dispersa) a un DataFrame de Pandas
# Usamos .toarray() para asegurarnos de que sea un array denso
X_processed_df = pd.DataFrame(X_processed.toarray(), columns=feature_names)

# 3. Mostrar las primeras filas y la info
print("\n--- Primeras 5 filas del DataFrame Procesado ---")
print(X_processed_df.head())

print(f"\n--- Información del DataFrame Procesado ---")
print(f"Forma original de X: {X.shape}")
print(f"Forma de X procesado (X_processed_df): {X_processed_df.shape}")
X_processed_df.info()

Creando DataFrame con datos procesados para visualización...

--- Primeras 5 filas del DataFrame Procesado ---
   num__INDICADOR_1  num__INDICADOR_2  num__INDICADOR_3  num__INDICADOR_4  \
0          0.437002         -0.556223          0.813978          0.060296   
1          0.346934         -0.481341          0.508180          0.016142   
2          0.232301         -0.492038          0.729034          0.016142   
3          1.771650         -0.941332         -0.171371         -1.072993   
4          0.387874         -0.299484          0.389259          0.457683   

   cat__E_PRGM_ACADEMICO_3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES  \
0                                                0.0                      
1                                                0.0                      
2                                                0.0                      
3                                                0.0                      
4                                                0.